In [4]:
from crewai import LLM
import os
import json
import re

In [5]:
# Load environment variables
causal_template = json.load(open("input/2_causal_interpretability.json"))

In [6]:
llm = LLM(

    #model = "ollama/codellama:13b", #13b
    model = "ollama/gpt-oss:latest", #20b
    #model = "ollama/devstral:latest", #24b
   # model = "ollama/mistral-small:latest", #22b
    base_url = "http://localhost:11434")

## Scenario A: Code generation text2code task

### Step 1: Define the task

In [7]:
messages_a = causal_template['scenario_a']['step1']['messages']

In [8]:
messages_a

[{'role': 'system',
  'content': 'You are an expert in causal inference and counterfactual reasoning. Your job is to infer a causal graph from a set of variables and draft counterfactual explanations.'},
 {'role': 'user',
  'content': '[CAUSAL TASK] Giving the following variable in the json file {FACTORS}, generate a causal graph that explains how they influence each other. Consider that those variables are influencing the generation of the function {FUNCTION}, therefore the code generation should be a separate node in the graph. The outcome must be in Graphviz DOT format.'},
 {'role': 'user',
  'content': '[COUNTERFACTUAL TASK] What would have changed or happened if the software developer had not considered the variable {NODE} during the generation of the {FUNCTION} function? Provide at most three counterfactual answers. Estimate a probability of occurrence for each answer based solely on the causal graph in Graphviz DOT format. The outcome must be contained in a json format in which 

In [6]:
#messages_a[1]['content'] =' '.join([messages_a[0]['content'], messages_a[1]['content']])

In [7]:
#messages_a[1]['content']

In [9]:
def extract_function_name(response):
    match = re.search(r"def\s+(\w+)\s*\(", response)
    if match:
        return match.group(1)
    return None

In [30]:
def process_messages(message,function, json_factors,node, test, summary):
    #new_message= re.sub(r"\[.*?\]","", message['content'])
    new_message= message['content']
    if function:
        new_message = re.sub(r"\{FUNCTION\}",function, new_message)
    if json_factors:
        new_message = re.sub(r"\{FACTORS\}",json_factors, new_message)
    if re.search(r"\{NODE\}", new_message):
        new_message = re.sub(r"\{NODE\}",node, new_message)
    if re.search(r"\{TEST\}", new_message):
        new_message = re.sub(r"\{TEST\}",test, new_message)
    if re.search(r"\{SUMMARY\}", new_message):
        new_message = re.sub(r"\{SUMMARY\}",summary, new_message)
       
    return new_message

In [11]:
def find_name(obj):
    if isinstance(obj, dict):
        for k, v in obj.items():
            if k == "name":
                yield v
            else:
                yield from find_name(v)
    elif isinstance(obj, list):
        for item in obj:
            yield from find_name(item)

In [12]:
def process_introspection_factors(introspection_chat_history):
    factors = []
    function = None
    function_message = introspection_chat_history[3]['content']
    function = extract_function_name(function_message)
    factors_message = introspection_chat_history[5]['content']
    cleaned = re.sub(r"^```json\s*|\s*```$", "", factors_message.strip(), flags=re.MULTILINE)

    data = json.loads(cleaned)
    factors=list(find_name(data))
    return function, function_message, factors, factors_message

In [15]:
import copy
chat_history = []
last_response = ''
messagges = copy.deepcopy(messages_a)
input_file= 'ollama_gpt-oss:latest_step1_step2_sptep3_15.json'
introspection_messaages =json.load(open("output/introspection/"+input_file))['scenario_a']['step1']['messages']

func, func_name, nodes, factors_message = process_introspection_factors(introspection_messaages)


first_node = nodes.pop(0)
last_message = {}
def run(message, node):
    
    message['content'] = process_messages(message, func, factors_message, node, None)
    print(message['content'])
    print("-----")
    chat_history.append(message)
    last_response = llm.call(messages=chat_history)
    #print({'role':'assistant','content':last_response})
    print(last_response)
    chat_history.append({"role": "assistant", "content": last_response})

for message in messagges:
    last_message = copy.deepcopy(message)
    run(message , first_node)

while nodes:
    print(last_message)
    run(last_message, nodes.pop(0))


You are an expert in causal inference and counterfactual reasoning. Your job is to infer a causal graph from a set of variables and draft counterfactual explanations.
-----
I’d love to help you sketch out the causal graph and write counterfactual explanations, but I need to know which variables you’re working with. Could you please list the variables (and any known relationships or constraints) you’d like me to analyze? Once I have that, I can infer the most plausible causal structure and draft the requested counterfactuals.
[CAUSAL TASK] Giving the following variable in the json file ```json
{
  "A": {
    "name": "Type hint usage",
    "description": "Adding explicit type hints (Sequence[float], List[float]) clarifies expected inputs and outputs, improving static analysis and developer experience.",
    "attribution_weight": 0.20
  },
  "B": {
    "name": "Input validation",
    "description": "Checking that both vectors have exactly three elements ensures the function behaves correc

In [16]:
chat_history


[{'role': 'system',
  'content': 'You are an expert in causal inference and counterfactual reasoning. Your job is to infer a causal graph from a set of variables and draft counterfactual explanations.'},
 {'role': 'assistant',
  'content': 'I’d love to help you sketch out the causal graph and write counterfactual explanations, but I need to know which variables you’re working with. Could you please list the variables (and any known relationships or constraints) you’d like me to analyze? Once I have that, I can infer the most plausible causal structure and draft the requested counterfactuals.'},
 {'role': 'user',
  'content': '[CAUSAL TASK] Giving the following variable in the json file ```json\n{\n  "A": {\n    "name": "Type hint usage",\n    "description": "Adding explicit type hints (Sequence[float], List[float]) clarifies expected inputs and outputs, improving static analysis and developer experience.",\n    "attribution_weight": 0.20\n  },\n  "B": {\n    "name": "Input validation",

In [17]:
messages_a

[{'role': 'system',
  'content': 'You are an expert in causal inference and counterfactual reasoning. Your job is to infer a causal graph from a set of variables and draft counterfactual explanations.'},
 {'role': 'user',
  'content': '[CAUSAL TASK] Giving the following variable in the json file {FACTORS}, generate a causal graph that explains how they influence each other. Consider that those variables are influencing the generation of the function {FUNCTION}, therefore the code generation should be a separate node in the graph. The outcome must be in Graphviz DOT format.'},
 {'role': 'user',
  'content': '[COUNTERFACTUAL TASK] What would have changed or happened if the software developer had not considered the variable {NODE} during the generation of the {FUNCTION} function? Provide at most three counterfactual answers. Estimate a probability of occurrence for each answer based solely on the causal graph in Graphviz DOT format. The outcome must be contained in a json format in which 

In [18]:
from IPython import get_ipython

# Get the current execution count
exec_count = get_ipython().execution_count
print("This cell run number is:", exec_count)

This cell run number is: 18


In [19]:
output ={
    'scenario_a': {
        'step1': {'messages': chat_history}
    }
}
file = "output/interpretability/"+ llm.model.replace("/","_")+"_step1_"+ str(exec_count) +".json"
with open(file, "w") as f:
    json.dump(output, f, indent=4)  

# Scenario A - interpretability - step 2

In [20]:
messages_2 = causal_template['scenario_a']['step2']['messages']

In [21]:
messages_2

[{'role': 'system',
  'content': 'You are an expert in causal inference and counterfactual reasoning. Your job is to infer a causal graph from a set of variables and draft counterfactual explanations.'},
 {'role': 'user',
  'content': '[CAUSAL TASK] Giving the following variables in the json file {FACTORS}, generate a causal graph that explains how they influence each other. Consider that those variables are influencing the generation of the test case {TEST}, therefore the code generation should be a separate node in the graph. The outcome must be in Graphviz DOT format.'},
 {'role': 'user',
  'content': '[COUNTERFACTUAL TASK] What would have changed or happened if the software tester had not considered the variable {NODE} during the generation of the {TEST} test? Provide at most three counterfactual answers. Estimate a probability of occurrence for each answer based solely on the causal graph in Graphviz DOT format. The outcome must be contained in a json format in which the first lev

In [22]:
import copy
chat_history_2 = []
last_response = ''
messagges = copy.deepcopy(messages_2)
input_file= 'ollama_gpt-oss:latest_step1_step2_sptep3_15.json'
introspection_messaages =json.load(open("output/introspection/"+input_file))['scenario_a']['step2']['messages']

func_name, func, nodes, factors_message = process_introspection_factors(introspection_messaages)


first_node = nodes.pop(0)
last_message = {}
def run(message, node):
    
    message['content'] = process_messages(message, None, factors_message, node, func)
    print(message['content'])
    print("-----")
    chat_history_2.append(message)
    last_response = llm.call(messages=chat_history_2)
    #print({'role':'assistant','content':last_response})
    print(last_response)
    chat_history_2.append({"role": "assistant", "content": last_response})

for message in messagges:
    last_message = copy.deepcopy(message)
    run(message , first_node)

while nodes:
    print(last_message)
    run(copy.deepcopy(last_message), nodes.pop(0))


You are an expert in causal inference and counterfactual reasoning. Your job is to infer a causal graph from a set of variables and draft counterfactual explanations.
-----


I’m ready to help you build the causal graph and draft counterfactual explanations, but I’ll need a bit more detail to get started. Specifically:

1. **List of Variables** – What are the observed variables (e.g., X₁, X₂, … Xₙ)?  
2. **Domain Context** – A short description of the problem setting (e.g., clinical study, marketing campaign, economic policy).  
3. **Data Availability** – Do you have observational data, experimental data, or both?  
4. **Any Prior Knowledge** – Known causal relationships or constraints (e.g., “age” cannot be caused by “income”).  
5. **Goal of Counterfactuals** – What kind of counterfactual questions do you want to answer? (e.g., “What if patient A had received drug B?”)

Once I have this information, I can:

- Sketch a plausible directed acyclic graph (DAG) or structural causal model (SCM).  
- Discuss identification assumptions (confounding, back‑door paths, instrumental variables, etc.).  
- Provide a step‑by‑step construction of counterfactuals using th

chat_history_2

In [23]:
chat_history_2

[{'role': 'system',
  'content': 'You are an expert in causal inference and counterfactual reasoning. Your job is to infer a causal graph from a set of variables and draft counterfactual explanations.'},
 {'role': 'assistant',
  'content': 'I’m ready to help you build the causal graph and draft counterfactual explanations, but I’ll need a bit more detail to get started. Specifically:\n\n1. **List of Variables** – What are the observed variables (e.g.,\u202fX₁,\u202fX₂,\u202f…\u202fXₙ)?  \n2. **Domain Context** – A short description of the problem setting (e.g.,\u202fclinical study, marketing campaign, economic policy).  \n3. **Data Availability** – Do you have observational data, experimental data, or both?  \n4. **Any Prior Knowledge** – Known causal relationships or constraints (e.g.,\u202f“age” cannot be caused by “income”).  \n5. **Goal of Counterfactuals** – What kind of counterfactual questions do you want to answer? (e.g.,\u202f“What if patient\u202fA had received drug\u202fB?”)

# Scenario A - interpretability - step 3

In [24]:
messages_3 = causal_template['scenario_a']['step3']['messages']

In [25]:
messages_3

[{'role': 'system',
  'content': 'You are an expert in causal inference and counterfactual reasoning. Your job is to infer a causal graph from a set of variables and draft counterfactual explanations.'},
 {'role': 'user',
  'content': '[CAUSAL TASK] Giving the following variables in the json file {FACTORS}, generate a causal graph that explains how they influence each other. Consider that those variables are influencing the generation of the summary {SUMMARY}, therefore the code generation should be a separate node in the graph. The outcome must be in Graphviz DOT format.'},
 {'role': 'user',
  'content': '[COUNTERFACTUAL TASK] What would have changed or happened if the software developer had not considered the variable {NODE} during the generation of the summary: {SUMMARY}? Provide at most three counterfactual answers. Estimate a probability of occurrence for each answer based solely on the causal graph in Graphviz DOT format. The outcome must be contained in a json format in which th

In [ ]:
def process_introspection_factors_3(introspection_chat_history):
    factors = []
    function = None
    function_message = introspection_chat_history[3]['content']
    function = extract_function_name(function_message)
    factors_message = introspection_chat_history[5]['content']
    cleaned = re.sub(r"^```json\s*|\s*```$", "", factors_message.strip(), flags=re.MULTILINE)

    data = json.loads(cleaned)
    factors=list(find_name(data))
    return function, function_message, factors, factors_message

In [26]:
import copy
chat_history_3 = []
last_response = ''
messagges = copy.deepcopy(messages_3)
input_file= 'ollama_gpt-oss:latest_step1_step2_sptep3_15.json'
introspection_messaages =json.load(open("output/introspection/"+input_file))['scenario_a']['step3']['messages']

_, summary, nodes, factors_message = process_introspection_factors(introspection_messaages)




In [ ]:
first_node = nodes.pop(0)
last_message = {}
def run(message, node):
    
    message['content'] = process_messages(message, None, factors_message, node, None, summary)
    print(message['content'])
    print("-----")
    chat_history_3.append(message)
    last_response = llm.call(messages=chat_history_3)
    #print({'role':'assistant','content':last_response})
    print(last_response)
    chat_history_3.append({"role": "assistant", "content": last_response})

for message in messagges:
    last_message = copy.deepcopy(message)
    run(message , first_node)

while nodes:
    print(last_message)
    run(copy.deepcopy(last_message), nodes.pop(0))

You are an expert in causal inference and counterfactual reasoning. Your job is to infer a causal graph from a set of variables and draft counterfactual explanations.
-----
Sure! To get started, could you please share:

1. **The list of variables** you’re interested in (e.g., “age”, “income”, “treatment”, “outcome”, etc.).
2. **Any data you have** (even a small sample or summary statistics) or at least a description of how these variables are measured and related in the real world.
3. **Your specific goal** for the counterfactuals (e.g., “What would happen to outcome X if we intervened on Y?”).
4. **Any domain constraints** or prior knowledge you already have (e.g., known causal direction, temporal ordering, or impossibilities).

Once I have that, I can:
- Construct a plausible causal graph (directed acyclic graph) based on the information and your domain knowledge.
- Identify the back‑door, front‑door, or instrumental variables needed.
- Draft counterfactual explanations for the scena

In [ ]:
output ={
    'scenario_a': {
        'step1': {'messages': chat_history},
        'step2': {'messages': chat_history_2},
        'step3': {'messages': chat_history_3}
    }
}
file = "output/interpretability/"+ llm.model.replace("/","_")+"_step1__step2_step3_"+ str(exec_count) +".json"
with open(file, "w") as f:
    json.dump(output, f, indent=4)  